In [2]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


# 1) Carga tu CSV original
csvFile = r'C:\Users\HP\Desktop\tfg ulti\data\Twitter_Data.csv';
df = pd.read_csv(csvFile)

# 2) Inicializa VADER
analyzer = SentimentIntensityAnalyzer()

# 3) Aplica al texto limpio
df['sentiment_cont'] = df['clean_text'].apply(
    lambda txt: analyzer.polarity_scores(str(txt))['compound']
)

# 4) Quédanos sólo con la columna numérica
df_num = df[['sentiment_cont']]

# 5) Guarda el CSV de sólo scores
output_path = r'C:\Users\HP\Desktop\tfg ulti\data\Twitter_Data_scores.csv'
df_num.to_csv(output_path, index=False)

print(df_num.head())

   sentiment_cont
0          0.5267
1         -0.4019
2          0.7096
3         -0.0713
4          0.4754
